This notebook applies SPARQL query to retrieve and download biomedically relevant edge category types from Wikidata to be utilized by the downstream drug repurposing algorithm.

I. [Load Packages](#Load) [clicking on phrase will take you directly to section] <br>
II. [Infectious Taxa as Part of Network](#Taxa) <br>


III. [Query for Biomedical Edge Types in Wikidata](#Query) <br>
IV. [Concatenate Node Types and Save as .csv](#Concatenate) <br>

## Load 
Packages and modules with relevant functions

In [5]:
%matplotlib inline 
# why is above line needed?
import pandas as pd

import functools # what does this do?
from pathlib import Path
from itertools import chain # what does this do?
from tqdm.autonotebook import tqdm 

from data_tools.df_processing import char_combine_iter, add_curi
from data_tools.plotting import count_plot_h
from data_tools.wiki import execute_sparql_query, node_query_pipeline, standardize_nodes, standardize_edges

<ipython-input-5-820a0019ca1f>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
def process_taxa(edges): # Integrate process_taxa() function into data_tools package ?
    nodes = edges.drop_duplicates(subset=['taxon', 'tax_id'])[['taxon', 'taxonLabel', 'tax_id']]
    nodes = add_curi(nodes, {'tax_id': 'NCBITaxon'})
    return standardize_nodes(nodes, 'taxon')

In [8]:
# What is happening in this code cell?
# Why do we need nodes to get edges? Is it a good idea that we have them?

prev_dir = Path('../results/').resolve()
prev_nodes = pd.read_csv(prev_dir.joinpath('01a_nodes.csv')) 

In [9]:
nodes = []
edges = []

## Taxa
We will account for the various taxa involved in or related to disease. This will include 2 types of syntax, and 2 approaches.

#### Syntax in the WikiData data model

1. Direct statements:  
    Taxon has-effect Disease... or Disease has-cause Taxon 
    

2. Qualifier Statements:  
    Disease has-cause infection (qual: of Taxon) 

#### Approaches  in the WikiData data model
1. Direct links:      
    Taxon has-effect Disease
    

2. Punning down to a specific taxonomic level:  
    Partent_taxon has-effect Disease  
    Taxon has-parent* Parent_taxon  
    Taxon has-rank Species 

In [12]:
# Direct statement: Disease causes infection
q = """SELECT DISTINCT ?disease ?taxon ?taxonLabel ?tax_id
    WHERE {
     {?disease wdt:P31 wd:Q12136}UNION{?disease wdt:P699 ?doid}.
      ?disease p:P828 [ps:P828 wd:Q166231;pq:P642 ?taxon;].
      OPTIONAL{?taxon wdt:P685 ?tax_id}.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}"""

qr = execute_sparql_query(q)
tax_nodes = process_taxa(qr)
edge_res = standardize_edges(qr, 'taxon', 'disease', 'causes')

nodes.append(tax_nodes)
edges.append(edge_res)

edge_res.head()

,start_id,end_id,type,dsrc_type
0,Q178235,Q18553238,causes,crowd_sourced
1,Q18968316,Q19000401,causes,crowd_sourced
2,Q9094478,Q55009681,causes,crowd_sourced
3,Q5348630,Q5348632,causes,crowd_sourced
4,Q15632444,Q55009423,causes,crowd_sourced


In [13]:
tax_nodes.head()

,id,name,label,xrefs
0,Q1010102,Bunyaviridae,Taxon,
1,Q1026758,Caliciviridae,Taxon,NCBITaxon:11974
2,Q10457821,Conidiobolus coronatus,Taxon,NCBITaxon:34488
3,Q10462705,Cryptostroma corticale,Taxon,NCBITaxon:1463975
4,Q10584816,Microsporum distortum,Taxon,NCBITaxon:101202


## Query
Biomedically relevant edge types in Wikidata (ordered alphabetically) <br>
To affirm a edge type category has been added, move it to its own cell and view separately using the 'print' function.

## Taxon Causes Disease (2 syntax, 2 approches) *why this part?

Theyre are two syntax for taxon causes disease that exist in the WikiData data model:

1. Direct statements:  
    Taxon has-effect Disease... or Disease has-cause Taxon 


2. Qualifier Statements:  
    Disease has-cause infection (qual: of Taxon) 
    
There are also two approaches:

1. Direct links:      
    Taxon has-effect Disease


2. Punning down to a specific taxonomic level:  
    Partent_taxon has-effect Disease  
    Taxon has-parent* Parent_taxon  
    Taxon has-rank Species 

### Syntax 1: disease has-cause infection - qualifier: of TAXON

In [ ]:
q = """    SELECT DISTINCT ?disease ?taxon ?taxonLabel ?tax_id
    WHERE 
    {
     {?disease wdt:P31 wd:Q12136}UNION{?disease wdt:P699 ?doid}.

     ?disease p:P828 [ps:P828 wd:Q166231;
                       pq:P642 ?taxon;].
      OPTIONAL{?taxon wdt:P685 ?tax_id}.
          
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }"""

In [ ]:
qr = execute_sparql_query(q)
tax_nodes = process_taxa(qr)
edge_res = standardize_edges(qr, 'taxon', 'disease', 'causes')

nodes.append(tax_nodes)
edges.append(edge_res)

edge_res.head()